In [13]:
using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
using SatellitePlayground
using SatelliteDynamics
using Plots
using LinearAlgebra
SP = SatellitePlayground

  Activating project at `~/Downloads/cmu_classes/16745/final_project`


SatellitePlayground

In [14]:
include("nominal_dynamics.jl")
include("rollout.jl")

generate_jacobians (generic function with 1 method)

In [15]:
J = SP.default_parameters.J
x0 = SP.initialize_orbit()
x0 = SP.RBState(
    x0.position,
    x0.velocity,
    [1, 0, 0, 0],
    zeros(3)
)

time = 60 * 60 # 1 hour
dt = 0.5
steps = time / dt
(states, times) = rollout(x0, steps, dt)
ref_traj = [
    SP.RBState(
        attitude=x0.attitude,
        angular_velocity=x0.angular_velocity,
        position=state.position,
        velocity=state.velocity
    )
    for state in states
    
]
@time As, Bs = generate_jacobians(ref_traj, times, J, dt)

intialized orbit!


[202/7200]: norm(ω)=0.000 r=<0.179 7255941.349 853170.348> b=<-0.059 -0.262 21.094> dt=0.500

[307/7200]: norm(ω)=0.000 r=<0.398 7206108.800 1293687.668> b=<-0.207 -4.635 20.951> dt=0.500

[412/7200]: norm(ω)=0.000 r=<0.686 7136090.074 1730573.730> b=<-0.324 -8.931 20.012> dt=0.500

[516/7200]: norm(ω)=0.000 r=<1.025 7047207.596 2158557.072> b=<-0.413 -12.908 18.359> dt=0.500

...

[2042/7200]: norm(ω)=0.000 r=<7.480 3936080.201 7344333.611> b=<-0.144 -17.012 -17.148> dt=0.500

[5251/7200]: norm(ω)=0.000 r=<2.094 -5934797.453 9723449.931> b=<0.514 7.196 -6.161> dt=0.500000

Excessive output truncated after 524376 bytes.

[5295/7200]: norm(ω)=0.000 r=<1.877 -6055663.991 9692834.440> b=<0.513 7.198 -5.946> dt=0.500

(Any[[1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.25 … 1.0 0.0; 0.0 0.0 … 0.0 1.0], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.25 … 1.0 0.0; 0.0 0.0 … 0.0 1.0], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.25 … 1.0 0.0; 0.0 0.0 … 0.0 1.0], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.25 … 1.0 0.0; 0.0 0.0 … 0.0 1.0], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.25 … 1.0 0.0; 0.0 0.0 … 0.0 1.0], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.25 … 1.0 0.0; 0.0 0.0 … 0.0 1.0], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.25 … 1.0 0.0; 0.0 0.0 … 0.0 1.0], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.25 … 1.0 0.0; 0.0 0.0 … 0.0 1.0], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.25 … 1.0 0.0; 0.0 0.0 … 0.0 1.0], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.25 … 1.0 0.0; 0.0 0.0 … 0.0 1.0]  …  [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.25 … 1.0 0.0; 0.0 0.0 … 0.0 1.0], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.25 … 1.0 0.0; 0.0 0.0 … 0.0 1.

In [16]:
function hat(ω)
    # TODO: implement the hat function 
    ω̂ = [0 -ω[3] ω[2]
        ω[3] 0 -ω[1]
        -ω[2] ω[1] 0]
    
    return ω̂
end

function rptoq(ϕ)
    (1/sqrt(1+ϕ'*ϕ))*[1; ϕ]
end
function qtorp(q)
    q[2:4]/q[1]
end

function L(q)
    # TODO: implement L 
    #_L = zeros(4,4)
    _L = [q[1] -q[2:end]'
        q[2:end] q[1]*I + hat(q[2:end])]
    return _L
end

function R(q)
    # TODO: implement R 
    #_R = zeros(4,4)
    _R = [q[1] -q[2:end]'
        q[2:end] q[1]*I - hat(q[2:end])]
    return _R
end

# TODO: implement H function 
H = zeros(4,3)
H = [0 0 0
    1 0 0
    0 1 0
    0 0 1]

function G(q)
    q_mat = [-q[2] q[1] q[4] -q[3];
             -q[3] -q[4] q[1] q[2];
            -q[4] q[3] -q[2] q[1]]
    _G = [I zeros(3,4);
            zeros(3,3) q_mat]
    return _G
end

G (generic function with 1 method)

In [17]:
N = length(ref_traj) 

# TODO: use TVLQR to generate K's 

# use this for TVLQR tracking cost 
Q_lqr = diagm([1e2,1e2, 1e2,1,1,1])
Qf_lqr = 10*Q_lqr
R_lqr = 1e2*diagm(ones(3))
nu = 3
nx = 13

K = [zeros(nu,nx) for i=1:N-1]
S = [zeros(nx,nx) for i = 1:N]
S[N]=deepcopy(Qf_lqr)
for k = N-1:-1:1
    xk = ref_traj[k]
    qk = xk.attitude
    Gk = G(qk)
    A = Gk*As[k]*Gk'
    B = Gk*Bs[k]
    K[k] = (R_lqr + B'S[k+1] * B) \ (B' * S[k+1] * A)
    S[k] = Q_lqr + K[k]' * R_lqr * K[k] + inv(A - B * K[k]) * (S[k+1]) * (A - B * K[k])
end

LoadError: SingularException(4)

In [25]:
S[N-1]

6×6 Matrix{Float64}:
 1129.11      0.0       0.0    -34.3279    0.0       0.0
    0.0    1129.11      0.0      0.0     -34.3279    0.0
    0.0       0.0    1129.11     0.0       0.0     -34.3279
 -127.433     0.0       0.0     15.4456    0.0       0.0
    0.0    -127.433     0.0      0.0      15.4456    0.0
    0.0       0.0    -127.433    0.0       0.0      15.4456

In [12]:
function log_state(hist, state)
    push!(hist, state)
end
function tvlqr_control(measurement)
    (state, _) = measurement
    x = Vector([state.angular_velocity; state.attitude])
    return Control(clamp.(controller(x), -1e-1, 1e-1))
end
i = 0
function controller(x)
    
    q0 = x0.attitude
    q = x[4:7]
    ω = x[1:3]
    ϕ = qtorp(L(q0)'*q)
    
    Δx̃ = [ω-x0.angular_velocity; ϕ]
    @show (ω, x0.angular_velocity, ϕ)
    global i
    i+=1
    u = -K[i]*Δx̃
    return u
end
q_true = [1, 0.1, 0.1, 0.1]
q_true /= norm(q_true)
x_true = SP.RBState(
    x0.position,
    x0.velocity,
    q_true,
    [0.01, 0.0, 0.0]
)

# (hist, time) = SP.simulate(tvlqr_control, log_step=log_state, max_iterations=N-1, dt=dt,
#         initial_condition=x_true)
nx = 7
nu = 3
uhist = zeros(nu,N)
xhist = zeros(nx,N)
q_true = [0.9, 0.03, 0.03,0.05]
q_true /= norm(q_true)
xhist[:,1] = [x0.angular_velocity; q_true]
p = attitude_params([0,0,0], J)
for k = 1:(N-1)
    uhist[:,k] = controller(xhist[:,k])
    xhist[:,k+1] = rk4(p, xhist[:,k], uhist[:,k], Epoch(2020,1,10), dt, nominal_attitude_dynamics)
end
xhist[:,N]

(ω, x0.angular_velocity, ϕ) = ([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.03333333333333333, 0.03333333333333333, 0.05555555555555556])


LoadError: DimensionMismatch: second dimension of A, 13, does not match length of x, 6

In [8]:
w_hist = xhist[1:3, :]
q_hist = xhist[4:7, :]

LoadError: UndefVarError: xhist not defined

In [9]:
K[N-1]

3×6 Matrix{Float64}:
 0.579244  0.0       0.0       0.000723829  0.0          0.0
 0.0       0.579244  0.0       0.0          0.000723829  0.0
 0.0       0.0       0.579244  0.0          0.0          0.000723829

In [10]:
plot(w_hist', title="Angular Velocity", xlabel="Time (s)", ylabel="Angular Velocity (rad/s)", labels=["ω1" "ω2" "ω3" "||ω||"])

LoadError: UndefVarError: w_hist not defined

In [11]:
plot(q_hist', title="Attitude", xlabel="Time (s)", ylabel="Attitude", labels=["q1" "q2" "q3" "q4"])

LoadError: UndefVarError: q_hist not defined